### HU.2022.BICODE.BIGDATA.MPDESNUT.001

#### Base, exploración de datos y modelación probabilística de la reincidencia en desnutrición aguda, en niños de 0 a 5 años 

###### Complejidad:	Alta
###### Descripción:	
Desde las direcciones de Primera Infancia y Nutrición del ICBF, se requiere contar con el desarrollo de un modelo probabilístico que permita a partir de los análisis de los datos y de las fuentes asociadas a eventos de desnutrición aguda, predecir la probabilidad de reincidencia de desnutrición aguda, de aquellos niños y niñas que presentan esta condición y logran su recuperación y a partir de éste, determinar si pueden o no reincidir en desnutrición aguda moderada o aguda severa.

##### A.	Criterios relacionados con la conformación de la base y la construcción de la variable objetivo

El modelo desarrollado deberá ser enfocado en los tipos de desnutrición aguda moderada y desnutrición aguda severa, específica de la población objeto de análisis (niños y niñas de rango de 0 a 5 años), por debajo de la línea z-2 o z-3 de peso talla para la longitud/talla.

Para la identificación de este tipo de Desnutrición se debe tener en cuenta los siguientes parámetros, con los cuales deberá construirse la variable objetivo:  
  
  > Desnutrición Aguda Moderada   Peso/Talla (P/T)   < -2 a <= -3  
  > Desnutrición Aguda Severa      Peso/Talla (P/T)   < -3 

	
•	Para la medición y construcción de la variable objetivo a modelar, deben enlistarse los servicios donde el niño fue atendido y/o participa en seguimiento del proceso:   

  >	Dirección de Primera Infancia  
  >	Dirección de Nutrición

•	Para la selección de los registros con desnutrición aguda se debe tener en cuenta los siguientes criterios:  

  >	Solo tomar aquellos registros con flag cero (0)  
  >	Registros cuya edad en meses al momento de la valoración sea de 0 a 59 meses  
  >	Registros cuyo sexo sea diferente a Intersexual (I)  
  >	Registros cuya información en Variable Diagnostico Discapacidad: es diferente de Sindromedown, acondroplasia, parálisis cerebral  
  >	Comparando las tallas de la toma que se está revisando y la anterior: si la anterior es inferior, no tomar esos registros  
  >	No tener en cuenta aquellos registros que, a la fecha de valoración, el beneficiario tenga menos de 40 semanas.  

•	La variable objetivo deberá contener los tres (3) momentos de evolución en las mediciones antropométricas que intervienen en la definición de reincidencia:
  
  > Una primera identificación de desnutrición aguda.  
  > Una segunda medición que indica que la situación de desnutrición aguda fue superada.  
  > Una tercera medición que indica que de nuevo existe desnutrición aguda.  
  
•	La variable objetivo deberá estar atada a una ventana de tiempo de observación trimestral o semestral, según como se defina que hay mayor fortaleza en la calidad de los datos. 
•	Así mismo, el modelo deberá realizar estimaciones en el marco de esa misma ventana temporal elegida.

•	Las observaciones de mediciones antropométricas deberán surtir un proceso de estandarización entre Nutrición y Primera Infancia, dado que en uno u otros servicios se realizan de forma mensual o trimestral. Por lo tanto, deberá decidirse si, en el caso de las mediciones mensuales, se tomaría la primera, segunda o tercera medición. La recomendación técnica apunta hacia tomar la primera medición del trimestre en la que se identifique la desnutrición aguda, como punto inicial para la variable objetivo.

•	Deberá conformarse una primera base histórica con suficientes registros que permitan realizar la modelación de manera adecuada, dividiendo los sets entre entrenamiento, comprobación y validación.

Importar librerías

In [0]:
import os, shutil, sys
import pandas as pd

Definir punto de montaje para extraer los datos del DataLake.   
> Contenedor icbf-bronze, base de datos RubOnline

### Conexion con los datos
##### INS, SISBEN,Indicadores Externos, Cuéntame (RubOnline) y DANE

In [0]:
dbfsPath = "dbfs:/mnt/bronze/icbf-rubonline"
path = os.listdir(f"{dbfsPath.replace('dbfs:','/dbfs/')}")

In [0]:
#Cortes Consolidados INS
BASE_CONFIRMADOS_BICF = "dbfs:/mnt/gold/icbf-datalakegai/Externas/INS/CONSOLIDADOS/BASE_CONFIRMADOS_BICF.parquet"
BASE_CONFIRMADOS_BICF = spark.read.parquet(BASE_CONFIRMADOS_BICF)

BASE_CONFIRMADOS_ICBF = "dbfs:/mnt/gold/icbf-datalakegai/Externas/INS/CONSOLIDADOS/BASE_CONFIRMADOS_ICBF.parquet"
BASE_CONFIRMADOS_ICBF = spark.read.parquet(BASE_CONFIRMADOS_ICBF)

BASE_EVENTO_113 = "dbfs:/mnt/gold/icbf-datalakegai/Externas/INS/CONSOLIDADOS/BASE_EVENTO_113.parquet"
BASE_EVENTO_113 = spark.read.parquet(BASE_EVENTO_113)

BASE_PROBABLES = "dbfs:/mnt/gold/icbf-datalakegai/Externas/INS/CONSOLIDADOS/BASE_PROBABLES.parquet"
BASE_PROBABLES = spark.read.parquet(BASE_PROBABLES)

In [0]:
BASE_CONFIRMADOS_BICF.columns

Out[4]: ['cod_eve',
 'fec_not',
 'semana',
 'año',
 'cod_pre',
 'cod_sub',
 'pri_nom_',
 'seg_nom_',
 'pri_ape_',
 'seg_ape_',
 'tip_ide_',
 'num_ide_',
 'edad_',
 'uni_med_',
 'nacionali_',
 'nombre_nacionalidad',
 'sexo_',
 'cod_pais_o',
 'cod_dpto_o',
 'cod_mun_o',
 'area_',
 'localidad_',
 'cen_pobla_',
 'vereda_',
 'bar_ver_',
 'dir_res_',
 'ocupacion_',
 'tip_ss_',
 'cod_ase_',
 'per_etn_',
 'nom_grupo_',
 'estrato_',
 'gp_discapa',
 'gp_desplaz',
 'gp_migrant',
 'gp_carcela',
 'gp_gestan',
 'sem_ges_',
 'gp_indigen',
 'gp_pobicbf',
 'gp_mad_com',
 'gp_desmovi',
 'gp_psiquia',
 'gp_vic_vio',
 'gp_otros',
 'fuente_',
 'cod_pais_r',
 'cod_dpto_r',
 'cod_mun_r',
 'fec_con_',
 'ini_sin_',
 'tip_cas_',
 'pac_hos_',
 'fec_hos_',
 'con_fin_',
 'fec_def_',
 'ajuste_',
 'telefono_',
 'fecha_nto_',
 'cer_def_',
 'cbmte_',
 'uni_modif',
 'nuni_modif',
 'fec_arc_xl',
 'nom_dil_f_',
 'tel_dil_f_',
 'fec_aju_',
 'nit_upgd',
 'fm_fuerza',
 'fm_unidad',
 'fm_grado',
 'version',
 'sit_defunc',
 'nom_madre',
 'apel_madre',
 'tip_ide_ma',
 'num_ide_ma',
 'edad_madre',
 'num_hi_viv',
 'num_hi_mue',
 'est_conyug',
 'ult_año_es',
 'mecanismo1',
 'mecanismo2',
 'mecanismo3',
 'mecanismo4',
 'asis_medic',
 'c_direct_a',
 'c_antecb',
 'c_antecc',
 'c_antecd',
 'otr_patol',
 'cau_pr_scm',
 'clas_caso',
 'ajuste',
 'fec_aju',
 'est_ingr',
 'nom_eve',
 'nom_upgd',
 'npai_proce',
 'ndep_proce',
 'nmun_proce',
 'npai_resi',
 'ndep_resi',
 'nmun_resi',
 'ndep_notif',
 'nmun_notif',
 'nreg',
 'edad_en meses',
 'clas_final',
 'empty_DG',
 'empty_DH',
 'empty_DI',
 'empty_DJ',
 'empty_DK',
 'empty_DL',
 'empty_DM',
 'empty_DN',
 'empty_DO',
 'empty_DP',
 'empty_DQ',
 'empty_DR',
 'empty_DS',
 'empty_DT',
 'empty_DE',
 'empty_DF',
 'empty_DU',
 'empty_DV',
 'empty_DW',
 'empty_DX',
 'empty_DY',
 'empty_DZ',
 'empty_DA',
 'edad en meses',
 'id_plan']

In [0]:
BASE_PROBABLES.columns

Out[5]: ['pri_nom_',
 'seg_nom_',
 'pri_ape_',
 'seg_ape_',
 'tip_ide_',
 'num_ide_',
 'fecha_nto_',
 'CRUZÓ_PARD',
 'MOTIVO_INGRESO_PARD',
 'FECHA_INGRESO_PARD',
 'REGIONAL_DIR_PARD',
 'CZ_DIR_PARD',
 'CRUZÓ_PI_TOMAS_2018',
 'FECHA_DE_VALORACIÓN_PI_TOMAS_2018',
 'ESTADO_PESO_TALLA_PI_TOMAS_2018',
 'FLAG_PI_TOMAS_2018',
 'SERVICIO_PI_TOMAS_2018',
 'EDAD_MESES_PI_TOMAS_2018',
 'CRUZÓ_PI_TOMAS_2019',
 'FECHA_DE_VALORACIÓN_PI_TOMAS_2019',
 'ESTADO_PESO_TALLA_PI_TOMAS_2019',
 'FLAG_PI_TOMAS_2019',
 'SERVICIO_PI_TOMAS_2019',
 'EDAD_MESES_PI_TOMAS_2019',
 'CRUZÓ_PI_2018',
 'FECHA_DE_ATENCIÓN_PI_2018',
 'SERVICIO_PI_2018',
 'EDAD_PI_2018',
 'CRUZÓ_PI_2019',
 'FECHA_DE_ATENCIÓN_PI_2019',
 'SERVICIO_PI_2019',
 'EDAD_PI_2019',
 'Nut_TomasRNECCentrosMilDias2018',
 'FECHA_DE_VALORACIÓN_Nut_TomasRNECCentrosMilDias2018',
 'ESTADO_PESO_TALLA_Nut_TomasRNECCentrosMilDias2018',
 'FLAG_Nut_TomasRNECCentrosMilDias2018',
 'EDAD_MESES_Nut_TomasRNECCentrosMilDias2018',
 'Nut_TomasRNECCentrosMilDias2019',
 'FECHA_DE_VALORACIÓN_Nut_TomasRNECCentrosMilDias2019',
 'ESTADO_PESO_TALLA_Nut_TomasRNECCentrosMilDias2019',
 'FLAG_Nut_TomasRNECCentrosMilDias2019',
 'EDAD_MESES_Nut_TomasRNECCentrosMilDias2019']

> Se enlistan las tablas (parquet) que se van a utilizar para este proceso.

In [0]:
listTables = [  
    
#Nutricion + Beneficiario
    "ben.infogrupoetnico",
    "ben.grupoetnico",
    "ben.regimenseguridadsocial",
    "ben.infoadicionaldiscapacidad",
    "ben.condiciondiscapacidad",
    "ben.saludnutricion",
    "ben.infogrupoetnico",
    "ben.grupoetnico",
    "ben.regimenseguridadsocial",
    "ben.infoadicionaldiscapacidad",
    "ben.condiciondiscapacidad",
    "ben.serviciovigenciasaludnutricion",
    "ben.atencionbeneficiario",
    "ben.tipobeneficiario",
    "ben.indicadornutricion",
    "ben.beneficiario",
    "con.serviciocontratado",

#Globales    
    
    "global.serviciovigencia",
    "global.tiposdocumentos",
    "global.sexo",
    "global.direccionesicbf",
        
#Demográficos
    
    "div.pais",
    "div.departamento",
    "div.municipio",
    "div.comuna",
    "div.barrio",
    "div.veredas",
    "div.rancheria"
]

> Función para montar las tablas:

In [0]:
notExists = []

for file in listTables: 
    newFile = file.split('.')[0] + '_' + file.split('.')[1] + '.parquet'
    tableName = newFile.replace('.parquet','')
    if newFile not in path: 
        notExists.append(file)
    else: 
        spark.read.parquet(f"{dbfsPath}/{newFile}").createOrReplaceTempView(f"{tableName}")
print(f"Las tablas: {notExists} no pudieron ser creadas")

Las tablas: [] no pudieron ser creadas

In [0]:
ins_cortes = spark.read.parquet("dbfs:/mnt/gold/icbf-gai/BasesComplementariasModeloProbabilistico/INS_CORTES.parquet",inferSchema="true",header="True")
ins_cortes.createOrReplaceTempView("ins_cortes")
#sisben = spark.read.format("delta").load("dbfs:/mnt/gold/icbf-datalakegai/Externas/DNP/Sisben_III_Consolidado/")
#
sisben = spark.read.options(header=True,delimiter='|').csv("dbfs:/mnt/gold/icbf-datalakegai/Externas/DNP/sisben4/cortes/2022/20220331/Sisben_20220331.csv")
sisben.createOrReplaceTempView("sisben")

In [0]:
len(sisben.columns)
len(ins_cortes.columns)
display(ins_cortes)
#INS_pri_ape_

INS_num_ide_,INS_cod_eve,INS_fec_not,INS_semana,INS_año,INS_cod_pre,INS_cod_sub,INS_pri_nom_,INS_seg_nom_,INS_pri_ape_,INS_seg_ape_,INS_tip_ide_,INS_edad_,INS_uni_med_,INS_nacionali_,INS_nombre_nacionalidad,INS_sexo_,INS_cod_pais_o,INS_cod_dpto_o,INS_cod_mun_o,INS_area_,INS_localidad_,INS_vereda_,INS_bar_ver_,INS_dir_res_,INS_ocupacion_,INS_tip_ss_,INS_cod_ase_,INS_per_etn_,INS_nom_grupo_,INS_estrato,INS_gp_discapa,INS_gp_desplaz,INS_gp_migrant,INS_gp_carcela,INS_gp_gestan,INS_gp_indigen,INS_gp_pobicbf,INS_gp_mad_com,INS_gp_desmovi,INS_gp_psiquia,INS_gp_vic_vio,INS_gp_otros,INS_fuente,INS_cod_dpto_r,INS_cod_mun_r,INS_fec_con_,INS_ini_sin_,INS_tip_cas_,INS_pac_hos_,INS_fec_hos_,INS_con_fin_,INS_fec_def_,INS_ajuste_,INS_telefono_,INS_fecha_nto_,INS_fec_arc_xl,INS_nom_dil_f_,INS_tel_dil_f_,INS_fec_aju_,INS_nit_upgd,INS_version,INS_pri_nom_ma,INS_seg_nom_ma,INS_pri_ape_ma,INS_seg_ape_ma,INS_tip_ide_ma,INS_num_ide_ma,INS_niv_educat,INS_menores,INS_peso_nac,INS_talla_nac,INS_edad_ges,INS_t_lechem,INS_e_complem,INS_crec_dllo,INS_esq_vac,INS_carne_vac,INS_peso_act,INS_talla_act,INS_per_braqui,INS_imc,INS_zscore_pt,INS_clas_peso,INS_zscore_te,INS_clas_talla,INS_edema,INS_delgadez,INS_piel_rese,INS_hiperpigm,INS_cambios_cabello,INS_palidez,INS_ruta_atenc,INS_nom_eve,INS_nom_upgd,INS_ndep_proce,INS_nmun_proce,INS_ndep_resi,INS_nmun_resi,INS_ndep_notif,INS_nmun_notif,INS_nreg,INS_EDAD_MESES,INS_CONCATENAR
null,113,2019-01-05,1,2019,5400190000,15,SIXTO,FINOL,CORONA,null,MS,1,1,null,null,M,862,1,850,3,null,9999999 SIN INFORMACION,null,BARRIO NUEVO ESCOBAL ASENTAMIENTO YUKPA,9999,N,null,1,YUKPA,1,2,2,1,2,null,2,2,null,2,2,2,2,1,54,1,2019-01-04,2018-07-01,4,2,- -,1,- -,0,0000,2017-11-06,2019-05-22,MAYLEM MÉNDEZ VERA,3183955853,2019-01-05,5.40019000015E11,SIVIGILA - 2018 -18.1.5$0$1.3,DIONISTA,null,CORONA,null,AS,AS,5,2,0,0.0,0,9,3,2,2,2,4.2,59.0,8.8,12.07,-3.8632,1,-7.386,1,2,1,1,2,1,1,1,DESNUTRICIÓN AGUDA EN MENORES DE 5 AÑOS,ACCION CONTRA EL HAMBRE,DEPTO DESCONOCIDO,VENEZUELA - SIN ESPECIFICACIÓN ADICIONAL DE LUGAR,NORTE SANTANDER,CUCUTA,NORTE SANTANDER,CUCUTA,1428,28.511943896559295,null
1,113,2019-02-18,8,2019,2000101794,1,DELLIA,null,IZQUIERDO,ARROYO,MS,1,1,170,COLOMBIA,F,170,20,1,1,NA,null,ASENTAMIENTO SEIVAN,ASENTAMIENTO SEIVAN,9999,N,null,1,YUKPA,null,2,2,2,2,2,2,2,2,2,2,2,1,1,20,1,2019-02-18,2019-02-13,4,1,2019-02-18,1,- -,7,SIN INFORMACION,2017-04-30,2019-05-22,EDGARDO MAESTRE,3202469415,2019-04-05,9.006010527E9,SIVIGILA - 2018 -18.1.5$0$2.2,SEWIA,null,ARROYO,BILLAFAÑA,CC,1133601023,5,1,0,0.0,0,0,0,2,3,2,8.4,80.0,12.0,13.13,-2.1232,2,-1.195,2,2,1,2,2,2,2,2,DESNUTRICIÓN AGUDA EN MENORES DE 5 AÑOS,UNIDAD PEDIATRICA SIMION BOLIVAR,CESAR,VALLEDUPAR,CESAR,VALLEDUPAR,CESAR,VALLEDUPAR,7160,8.382642998027599,null
2,113,2019-06-15,24,2019,5400190000,13,URIEL,null,RODRIGUEZ,GUTIERREZ,MS,null,null,862,VENEZUELA,M,170,54,1,1,CUCUTA,null,VALLES DEL RODEO,VALLES DEL RODEO MZ B CASA 2,9999,N,null,6,null,2,2,2,1,2,null,2,2,null,2,2,2,2,1,54,1,2019-06-15,2019-06-01,4,2,- -,1,- -,0,3209889518,2017-06-26,2019-06-19,GUSTAVO GONZALEZ,5784980,2019-06-17,5.40019000013E11,SIVIGILA - 2018 -18.1.5$0$2.3,KATIUSCA,ALEXANDRA,GUTIERREZ,null,CE,18699527,4,1,2960,49.0,38,0,10,2,3,2,10.0,87.0,0.0,13.21,-2.3123,2,0.0197,3,2,1,2,2,2,2,1,DESNUTRICIÓN AGUDA EN MENORES DE 5 AÑOS,AMERICARES IPS BOCONO,NORTE SANTANDER,CUCUTA,NORTE SANTANDER,CUCUTA,NORTE SANTANDER,CUCUTA,11133,23.635765943458203,null
3,113,2019-02-22,8,2019,8100190001,0,JESUS,ESNEIDER,ROA,CASTRO,MS,3,1,862,VENEZUELA,M,862,1,850,1,NA,null,GUASDUALITO,GUASDUALITO VENEZUELA,9999,I,null,6,null,1,2,2,1,2,null,2,2,null,2,2,2,2,1,1,850,2019-02-22,- -,4,2,- -,1,- -,0,NO REFIERE,2015-12-17,2019-05-22,VANESSA HERNANDEZ,3103164260,2019-02-25,8.921201151E9,SIVIGILA - 2018 -18.1.5$0$0.2,LAURA,SORELIS,CASTRO,GALINDEZ,AS,VE26351032,2,2,3700,47.0,37,0,6,2,2,1,10.0,91.0,0.0,12.08,-3.5518,1,-1.6787,2,2,1,1,1,1,1,2,DESNUTRICIÓN AGUDA EN MENORES DE 5 AÑOS,FUNDACION AMERICARES,DEPTO DESCONOCIDO,VENEZUELA - SIN ESPECIFICACIÓN A

Importación de datos de tabla de indicadores externos más los metadatos para complementar la base principal

In [0]:
#========================================================

#             Tabla de indicadores externos

# COD   CICLO VITAL 

# 1     Primera Infancia 
# 2     Infancia
# 3     Adolescencia 
# 4     Infancia, niñez y adolescencia
# 5     Total Pobalción

#========================================================

# COD 
# TIPO         TIPO      
# ENTIDAD      ENTIDAD 
# TERRITORIAL  TERRITORIAL 

# 1            Municipio 
# 2            Departamento 
# 3            Nacional

#========================================================
consolidadoIndicadores = spark.read.parquet("dbfs:/mnt/gold/icbf-datalakegai/Internas/base indicadores/cortes/2021/CONSOLIDADO_INDICADORES_2019.parquet",inferSchema="true",header="True")
consolidadoIndicadores.createOrReplaceTempView("consolidadoIndicadores")

metadadatosIndicadores = spark.read.parquet("dbfs:/mnt/gold/icbf-datalakegai/Internas/base indicadores/cortes/2021/METADATOS INDICADORES.parquet",inferSchema="true",header="True")
metadadatosIndicadores.createOrReplaceTempView("metadadatosIndicadores")

Creación de DF de tabla de indicadores externos + metadatos de indicadores externos

In [0]:
TablaIndicadores = spark.sql("""
SELECT 
  --- Indicadores
   CI.TIPOENTIDADTERRITORIAL
  ,CI.CODDANEDEPARTAMENTO
  ,CI.CODDANEMUNICIPIO
  ,CI.CODINDICADOR
  ,CI.NOMBREINDICADOR
  ,CI.CICLOVITAL
  ,CI.RANGO
  ,CI.ANO
  ,CI.MES
  ,CI.VALOR 
  --- Metadatos
  ,MI.DIMENSION
  ,MI.PERIODICIDAD
  ,MI.SECTOR
  ,MI.DESCRIPCION
  ,MI.TIPODEVALOR
  ,MI.SUIN
  ,MI.PND
  ,MI.FUENTE  

FROM consolidadoIndicadores CI
LEFT JOIN metadadatosIndicadores MI 
ON MI.CODINDICADOR = CI.CODINDICADOR
""")
TablaIndicadores.createOrReplaceTempView("TablaIndicadores")

In [0]:
len(TablaIndicadores.columns)

Out[12]: 18

Generamos un consolidado de la vigencia de los servicios de ben_ServicioVigenciaSaludNutricion

In [0]:
ServicioVigenciaSaludNutricion_t1 = spark.sql("""
select 
	 svsn.IdSaludNutricion,
	 svsn.IdServicioVigencia,
	 row_number() over (partition by svsn.IdSaludNutricion order by svsn.fechacrea desc) rno
from ben_ServicioVigenciaSaludNutricion svsn
""")
ServicioVigenciaSaludNutricion_t1.createOrReplaceTempView("ServicioVigenciaSaludNutricion_t1")

In [0]:
len(ServicioVigenciaSaludNutricion_t1.columns)

Out[14]: 3

Filtrado de datos por incapaciadad S.D, Paralisis cerebral

In [0]:
#Estos son los discapacitados a excluir:
discapacidadExcluir = spark.sql("""
select 
    IdBeneficiario,
    UPPER(d.Descripcion)  Descripcionfrom 
FROM ben_condiciondiscapacidad d 
where 
    UPPER(d.Descripcion) like '%DOWN%'
    OR UPPER(d.Descripcion) like '%DAWN%'
    OR UPPER(d.Descripcion) like '%DAÇWM%'
    OR UPPER(d.Descripcion) like '%ACONDROPLASIA%'
    OR UPPER(d.Descripcion) like '%LISIS CEREBR%'
    OR UPPER(d.Descripcion) like '%LICIS CEREBRAL%'
    OR UPPER(d.Descripcion) like '%LISIS SEREBRAL%'
    OR UPPER(d.Descripcion) like '%LISIS CELEBRAL%'
""")
discapacidadExcluir.createOrReplaceTempView("discapacidadExcluir")

In [0]:
print("Total de posibles discapacidades a excluir: ",discapacidadExcluir.count())

Total de posibles discapacidades a excluir: 787

Consolidado de discapacitados con exclusiones por c/u  de las atenciones: idbeneficiario | idatencionbeneficiario | presenta_discapacidad

In [0]:
discapacidad = spark.sql("""
select
    ad.idbeneficiario
	,ad.idatencionbeneficiario 
	,case 
		when count(ad.idbeneficiario) > 1 then 'SI'
		else 'NO'
	end as presenta_discapacidad
from ben_infoadicionaldiscapacidad ad
left join ben_condiciondiscapacidad as d 
on ad.idinfoadicionaldiscapacidad = d.idinfoadicionaldiscapacidad
where
    1=1
    and ad.requiereayudatecnica in ('S', 'N')                                                                ---57.057
    and ad.idbeneficiario not in (select distinct(IdBeneficiario) from discapacidadExcluir)                  ---56.636
group by ad.idbeneficiario,ad.idatencionbeneficiario 
""")
discapacidad.createOrReplaceTempView("discapacidad")

In [0]:
print("Total de discapacitados a excluir: ",discapacidad.count())

Total de discapacitados a excluir: 87572

A continuación se genera la primera base, con los campos solicitados.

Los INNER JOIN permiten asegurar que las observaciones de ben_indicadorNutricion:

> Tenga un beneficiario relacionado en ben_Beneficiario.  
> Tengan datos complementarios de ben_SaludNutricion.  
> Tenga un servicio que si haya tenido vigencia en ServicioVigenciaSaludNutricion_t1.   
> Tenga un servicio vigente y que exista en global_ServicioVigencia.  

Esto pues en el ejercicio de consolidación se identificaron algunas particularidades y a través de estos cruces se asegura la congruencia de los registros.

Los LEFT JOIN permiten agregar datos complementarios a cada uno de los registros
> Datos sociodemográficos

> A continuación se detalla el primer proceso de filtrado de datos: Se excluyen beneficiarios no registrados, se excluyen tomas de beneficiarios que no registran en Salud Nutrición, se excluyen los registros que no tienen asociado un servicio que haya estado o que esté vigente en Cuéntame: ServicioVigenciaSaludNutricion y por último un filtro para que todos los servicios deben existir si o sí en el Global de Servicios

In [0]:
%sql
SELECT 
  COUNT(DISTINCT(nu.IdBeneficiario)) as Total_Beneficiarios_Unicos 
FROM ben_indicadorNutricion nu                                                                                                                                  ------ 7.664.133
INNER JOIN ben_Beneficiario be ON nu.IdBeneficiario = be.IdBeneficiario                                                                                         ------ 7.645.575  --> Se excluyen beneficiarios no registrados
INNER JOIN ben_SaludNutricion sn ON be.IdBeneficiario = sn.IdBeneficiario AND nu.IdSaludNutricion = sn.IdSaludNutricion                                         ------ 7.607.561  --> Se excluyen tomas de beneficiarios que no registran en Salud Nutrición
INNER JOIN ben_atencionbeneficiario a ON sn.idatencionbeneficiario = a.idatencionbeneficiario                                                                   ------ 7.597.529  --> Aseguramos que el niño pertenezca un servicio vigencia.
INNER JOIN con_serviciocontratado sc ON a.idserviciocontratado = sc.idserviciocontratado                                                                        ------ 7.597.529  --> Con esto aseguramos que el contrato tenga/haya tenido vigencia y un servicio
INNER JOIN global_serviciovigencia AS sv  ON SC.IdServicioVigencia=SV.IdServicioVigencia                                                                        ------ 7.597.529  --> Todos los servicios deben tener/tuvieron vigencia dentro del contrato

Total_Beneficiarios_Unicos
8114637


In [0]:
baseModelo_t1 = spark.sql("""
select 
	 year(sn.FechaValoracionNuricional) vigencia
	,nu.IdBeneficiario
	,case 
		when tba.nombretipobeneficiario is null or tba.nombretipobeneficiario = '0' then tbb.nombretipobeneficiario
		else tba.nombretipobeneficiario 
	end as tipobeneficiario
   
    ,DATE(be.FechaNacimiento) AS FechaNacimientoBeneficiario
    ,DATEDIFF(DATE(nu.FechaValoracion),DATE(be.FechaNacimiento)) as EdadBeneficiarioDias
    ,ROUND(DATEDIFF(DATE(NU.FechaValoracion),DATE(be.FechaNacimiento))/30,0) as EdadBeneficiarioMeses
    ,(YEAR(NU.FechaValoracion) - YEAR(be.FechaNacimiento))  AS EdadBeneficiarioAnios      
    ,CASE
        WHEN DATEDIFF(DATE(NU.FechaValoracion),DATE(be.FechaNacimiento)) < 280 THEN "< 40 Semanas"
        WHEN DATEDIFF(DATE(NU.FechaValoracion),DATE(be.FechaNacimiento)) >= 280 THEN "> 40 Semanas"
    ELSE "Validar"
    END AS SemanasBeneficiario    
    
	,td.CodDocumento
	,be.NumeroDocumento
	,be.PrimerNombre
	,be.SegundoNombre
	,be.PrimerApellido
	,be.SegundoApellido
	,be.FechaNacimiento
	,s.Codigo sexo
    ,s.Nombre NombreSexo
    ,(floor(datediff(be.fechanacimiento,sn.FechaValoracionNuricional)/30.4375)*-1) EdadMesesBenAlaTomaNut
    
	,pa.CodigoPais CodPaisNacimientoBeneficiario
    ,pa.NombrePais NombPaisNacimientoBeneficiario
	,PRB.CodigoPais CodPaisRasidenciaBen
    ,PRB.NombrePais NombPaisRasidenciaBen
    	
    ,DRB.CodigoDepartamento CodDepartamentoResidenciaBen
    ,DRB.NombreDepartamento NomDepartamentoResidenciaBen
    ,MRB.CodigoMunicipio CodMunicipResidenciaBen
    ,MRB.NombreMunicipio NomMunicipResidenciaBen
    
    ,CO.CodigoComuna CodComunaResidenciaBen
    ,CO.NombreComuna NomComunaResidenciaBen 
    
    ,BA.CodigoBarrio CodBarrioResidenciaBen
    ,BA.NombreBarrio NomBarrioResidenciaBen 

    ,VE.CodigoVereda CodVeredaResidenciaBen
    ,VE.NombreVereda NomVeredaResidenciaBen

    ,RB.IdRancheria CodRancheriaResidenciaBen
    ,RB.Nombre NomRancheriaResidenciaBen

    
    ,dis.presenta_discapacidad PresentaDiscapacidad
	,case 
		when be.ZonaUbicacion = 'C' then 'CABECERA'
		when be.ZonaUbicacion = 'R' then 'RESTO'
		else be.ZonaUbicacion
	end ZonaUbicacionBeneficiario
	,ge.Nombre GrupoEtnico
	,sn.LactanciaMaternaExclusiva MesesLactanciaMaternaExclusiva
	,sn.LactanciaMaternaTotal MesesLactanciaMaternaTotal
	,sn.PesoBeneficiarioAlNacer
	,sn.TallaBeneficiarioAlNacer
	,sn.AntecendentePremadurez 
	,sn.EdadGestacionarAlNacer
	,rss.Nombre RegimenSeguridadSocial
	,concat(year(sn.FechaValoracionNuricional),DATE_PART('month',sn.FechaValoracionNuricional)) Toma
	
    ,gdir.IdDireccionesICBF
    ,gdir.NombreDireccion as DireccionICBF
    ,sv.NombreServicio 
	
    ,sn.FechaValoracionNuricional
	,sn.FechaMedicionPerimetroBraquial 
	,sn.MedicionPerimetroBraquial
	,sn.peso
	,sn.Talla
	,nu.ZScoreTallaEdad
	,nu.ZScorePesoEdad
	,nu.ZScorePesoTalla
	,nu.ZScoreIMC
	,nu.EstadoTallaEdad
	,nu.EstadoPesoEdad 
	,nu.EstadoPesoTalla
	,nu.EstadoIMC
	,nu.Flag
	,sn.FechaCrea FechaRegistroSaludNutricion
	,sn.PresentaCarneVacunacion
	,sn.ControlesCrecimDesarrollo
--	,row_number() over(partition by nu.idbeneficiario order by sn.FechaValoracionNuricional) NoToma
FROM ben_indicadorNutricion nu                                                                                                                                  ------ 7.664.133
INNER JOIN ben_Beneficiario be ON nu.IdBeneficiario = be.IdBeneficiario                                                                                         ------ 7.645.575  --> Se excluyen beneficiarios no registrados
INNER JOIN ben_SaludNutricion sn ON be.IdBeneficiario = sn.IdBeneficiario AND nu.IdSaludNutricion = sn.IdSaludNutricion                                         ------ 7.607.561  --> Se excluyen tomas de beneficiarios que no registran en Salud Nutrición
INNER JOIN ben_atencionbeneficiario a ON sn.idatencionbeneficiario = a.idatencionbeneficiario                                                                   ------ 7.597.529  --> Aseguramos que el niño pertenezca un servicio vigencia.
INNER JOIN con_serviciocontratado sc ON a.idserviciocontratado = sc.idserviciocontratado                                                                        ------ 7.597.529  --> Con esto aseguramos que el contrato tenga/haya tenido vigencia y un servicio
INNER JOIN global_serviciovigencia AS sv  ON SC.IdServicioVigencia=SV.IdServicioVigencia                                                                        ------ 7.597.529  --> Todos los servicios deben tener/tuvieron vigencia dentro del contrato

LEFT JOIN DIV_Pais PRB on be.IdPaisResidencia=PRB.IdPais 
left join div_pais pa on be.IdPaisNacimiento = pa.IdPais
LEFT JOIN DIV_Departamento DRB on be.IdDepartamentoResidencia=DRB.IdDepartamento 
LEFT JOIN DIV_Municipio MRB on be.IdMunicipioResidencia=MRB.IdMunicipio
LEFT JOIN DIV_Comuna CO ON be.IdComuna=CO.IdComuna and MRB.IdMunicipio=CO.IdMunicipio     
LEFT JOIN DIV_Barrio BA  on be.IdBarrio = BA.IdBarrio
LEFT JOIN DIV_Veredas VE  on be.IdVereda = VE.IdVereda
LEFT JOIN DIV_Rancheria RB on be.IdRancheria=RB.IdRancheria

left join ben_AtencionBeneficiario ab on sn.IdAtencionBeneficiario = ab.IdAtencionBeneficiario 
left join ben_TipoBeneficiario tbb on be.IdTipoBeneficiario = tbb.IdTipoBeneficiario
left join ben_TipoBeneficiario tba on ab.IdTipoBeneficiario = tba.IdTipoBeneficiario
left join global_TiposDocumentos td on be.IdTipoDocumento = td.IdTipoDocumento
left join Global_Sexo s on be.IdSexo = s.IdSexo
left join global_direccionesicbf gdir on gdir.IdDireccionesICBF = sv.IdDireccionesICBF

-- Acá cruzamos los discapacitados cuya información en Variable Diagnostico Discapacidad: es diferente de Sindromedown, acondroplasia, parálisis cerebral
left join discapacidad dis on nu.IdBeneficiario = dis.IdBeneficiario and ab.IdAtencionBeneficiario = dis.IdAtencionBeneficiario
left join ben_infogrupoetnico ige on be.IdBeneficiario = ige.IdBeneficiario and ab.IdAtencionBeneficiario = ige.IdAtencionBeneficiario
left join ben_GrupoEtnico ge on ige.IdGrupoEtnico = ge.IdGrupoEtnico
left join ben_RegimenSeguridadSocial rss on sn.IdRegimenSeguridadSocial = rss.IdRegimenSeguridadSocial
""")
baseModelo_t1.createOrReplaceTempView("baseModelo_t1")

In [0]:
len(baseModelo_t1.columns)

Out[20]: 65

Una vez generada la primera base se establecen filtros para asegurar lo indicado en los criterios de aceptación:

> IdDireccionesICBF IN (1,4)                     -- (1)Dirección de Primera Infancia  y (4) DireccionICBF: "Dirección de Nutrición"  
> Flag = 0                                       -- Flag = 0  
> EdadMesesBenAlaTomaNut BETWEEN 0 and 59        -- Beneficiarios de 0 a 59 meses  
> sexo not like ('I')                            -- Excluir Intersexuales  
> SemanasBeneficiario not like ('< 40 Semanas')  -- Excluir Beneficiarios que tengan menos de cuarenta semanas a la fecha de la toma de la muestra

In [0]:
baseModelo_t1 = spark.sql("""
select 
    vigencia as AnioToma,
 -- row_number() over(partition by idbeneficiario order by FechaValoracionNuricional) NoToma,
    nvl(lag(Talla) over (partition by idbeneficiario order by FechaValoracionNuricional),0) tallaMuestraAnterior,
    RegimenSeguridadSocial,
    Toma,
    IdDireccionesICBF,
    DireccionICBF,
    NombreServicio,
    IdBeneficiario,
    FechaNacimientoBeneficiario,
    EdadBeneficiarioDias,
    EdadBeneficiarioMeses,
    EdadBeneficiarioAnios,
    SemanasBeneficiario,
    CodDocumento,
    NumeroDocumento,
    PrimerNombre,
    SegundoNombre,
    PrimerApellido,
    SegundoApellido,
    tipobeneficiario,
    sexo,
    NombreSexo,
    EdadMesesBenAlaTomaNut,
    CodPaisNacimientoBeneficiario,
    NombPaisNacimientoBeneficiario,
    CodPaisRasidenciaBen,
    NombPaisRasidenciaBen,
    CodDepartamentoResidenciaBen,
    NomDepartamentoResidenciaBen,
    CodMunicipResidenciaBen,
    NomMunicipResidenciaBen,
    CodComunaResidenciaBen,
    NomComunaResidenciaBen,
    CodBarrioResidenciaBen,
    NomBarrioResidenciaBen,
    CodVeredaResidenciaBen,
    NomVeredaResidenciaBen,
    CodRancheriaResidenciaBen,
    NomRancheriaResidenciaBen,
    PresentaDiscapacidad,
    ZonaUbicacionBeneficiario,
    GrupoEtnico,
    MesesLactanciaMaternaExclusiva,
    MesesLactanciaMaternaTotal,
    PesoBeneficiarioAlNacer,
    TallaBeneficiarioAlNacer,
    AntecendentePremadurez,
    EdadGestacionarAlNacer,
    FechaValoracionNuricional,
    FechaMedicionPerimetroBraquial,
    MedicionPerimetroBraquial,
    peso,
    Talla,
    ZScoreTallaEdad,
    ZScorePesoEdad,
    ZScorePesoTalla,
    ZScoreIMC,
    EstadoTallaEdad,
    EstadoPesoEdad,
    EstadoPesoTalla,
    EstadoIMC,
    Flag,
    FechaRegistroSaludNutricion,
    PresentaCarneVacunacion,
    ControlesCrecimDesarrollo
  from baseModelo_t1  
where 
  IdDireccionesICBF IN (1,4)                         -- (1)Dirección de Primera Infancia  y (4) DireccionICBF: "Dirección de Nutrición"
  AND Flag = 0                                       -- Flag = 0
  AND EdadMesesBenAlaTomaNut BETWEEN 0 and 59        -- Beneficiarios de 0 a 59 meses
  AND sexo not like ('I')                            -- Excluir Intersexuales
  AND SemanasBeneficiario not like ('< 40 Semanas')  -- Excluir Beneficiarios que tengan menos de cuarenta semanas a la fecha de la toma de la muestra
ORDER BY                                             -- Ordernar por Beneficiario, Año Toma y N° Toma
  IdBeneficiario, 
  AnioToma
  --NoToma 
""")
baseModelo_t1.createOrReplaceTempView("baseModelo_t1")

In [0]:
len(baseModelo_t1.columns)

Out[22]: 65

In [0]:
%sql
select 
  count(distinct(IdBeneficiario)) as DireccionICBF, 
  DireccionICBF
from baseModelo_t1 
group by DireccionICBF

DireccionICBF,DireccionICBF
6162382,Dirección de Primera Infancia
223472,Dirección de Nutrición


> Se valida que las tomas de la talla actual sea mayor o igual a la talla de la toma anterior

In [0]:
baseModelo_t1 = spark.sql("""
select 
    *
  from baseModelo_t1
  where talla >= tallaMuestraAnterior
  and AnioToma >=2017
""")
baseModelo_t1.createOrReplaceTempView("baseModelo_t1")

In [0]:
%sql
select 
  count(distinct(IdBeneficiario)) as TotalBenUnicos, 
  AnioToma
From baseModelo_t1 
group by AnioToma
order by AnioToma

TotalBenUnicos,AnioToma
1734605,2017
1772949,2018
1734837,2019
1019932,2020
1677386,2021
1311996,2022


> A continuación se detalla el segundo proceso de filtrado de datos: Incluir los beneficiarios que han tomado servicios de las direcciones (1)Dirección de Primera Infancia y (4)Dirección de Nutrición, excluir Beneficiarios que tengan menos de cuarenta semanas a la fecha de la toma de la muestra, excluir Intersexuales y excluir los beneficiarios cuyo rango de edad es superior a los 59 meses

Prueba para dos beneficiarios con más de una observación. En el segundo campo se evidencia el NoToma

In [0]:
%sql
--SELECT
--  * 
--FROM baseModelo_t1 
--WHERE idBeneficiario in (17013823, 18602100)

Se guarda esta base puesto que es el insumo principal de Beneficiario + Nutrición

In [0]:
baseModelo_t1 = spark.sql("""
select     
  row_number() over(partition by idbeneficiario order by FechaValoracionNuricional asc) NoToma,
  * 
from baseModelo_t1 
order by 
  idBeneficiario
  ,FechaValoracionNuricional asc
""")
baseModelo_t1.createOrReplaceTempView("baseModelo_t1")

In [0]:
%sql
--select * from baseModelo_t1 limit 1000

In [0]:
#---------------------------- Borrar Histórico ----------------------------------------#
dbfsWriteDelta1 = "dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilistico/"
try:
    dbutils.fs.ls(dbfsWriteDelta1)
    print(" *** Borrando Directorio *** ")
    files = dbutils.fs.ls(dbfsWriteDelta1)
    for f in files:
        f.path
        dbutils.fs.rm(f.path, recurse=True)
except:
    print(" *** No Existe Directorio *** ")
    pass
  
#---------------------------- Directorio de Salida ------------------------------------#
print(" *** Guardando en DL *** ")
pathSave = dbfsWriteDelta1
nombreTabla = "dbo.baseModelo_t1"
baseModelo_t1.write \
.format("delta") \
.mode("overwrite") \
.option("overwriteSchema", "true") \
.save(f"{pathSave}") 
print(" *** Fin Guardado en el DataLake *** ")
spark.sql(f"CREATE TABLE IF NOT EXISTS {nombreTabla} USING DELTA LOCATION '{pathSave}/'")
print(" *** Fin Creación Tabla *** ")

*** Borrando Directorio *** 
 *** Guardando en DL *** 
 *** Fin Guardado en el DataLake *** 
 *** Fin Creación Tabla ***

Se toman los datos del beneficiario + los datos de la primera toma, se pivotean los campos por cada toma y se crea un DF por cada una.

In [0]:
%sql
select 
  count(1) as TotalRegistros,
  NoToma
from baseModelo_t1
where NoToma <=17
group by NoToma
order by NoToma

TotalRegistros,NoToma
4392521,1
3870011,2
3317520,3
2811404,4
2208325,5
1814501,6
1516513,7
1239888,8
950097,9
736929,10


In [0]:
len(baseModelo_t1.columns)
#row = baseModelo_t1.count()
#row

Out[26]: 66

In [0]:
for toma in range(17):
    tomaReal = toma + 1
    if tomaReal != 1:
        print(f"-------- Inicio creacion tabla NoToma: {tomaReal}--------")
        tableName = f"Toma{tomaReal}"
        print(f"-- Paso1: Creacion dataframe Inicio -- ")
        globals()["Toma" + str(tomaReal)] = spark.sql(f"""
            select 
                 IdBeneficiario IdBeneficiarioT{tomaReal}
                ,Toma TomaT{tomaReal}
                ,NoToma NoTomaT{tomaReal}
                ,IdDireccionesICBF IdDireccionesICBFT{tomaReal}
                ,DireccionICBF DireccionICBFT{tomaReal}
                ,NombreServicio NombreServicioT{tomaReal}
                ,EdadBeneficiarioDias EdadBeneficiarioDiasT{tomaReal}
                ,FechaRegistroSaludNutricion FechaRegistroSaludNutricionT{tomaReal}
                ,FechaMedicionPerimetroBraquial FechaMedicionPerimetroBraquialT{tomaReal}
                ,MedicionPerimetroBraquial MedicionPerimetroBraquialT{tomaReal}
                ,peso pesoT{tomaReal}
                ,Talla TallaT{tomaReal}
                ,ZScoreTallaEdad ZScoreTallaEdadT{tomaReal}
                ,ZScorePesoEdad ZScorePesoEdadT{tomaReal}
                ,ZScorePesoTalla ZScorePesoTallaT{tomaReal}
                ,ZScoreIMC ZScoreIMCT{tomaReal}
                ,EstadoTallaEdad EstadoTallaEdadT{tomaReal}
                ,EstadoPesoEdad EstadoPesoEdadT{tomaReal}
                ,EstadoPesoTalla EstadoPesoTallaT{tomaReal}
                ,EstadoIMC EstadoIMCT{tomaReal}
                ,Flag FlagT{tomaReal}
                ,PresentaCarneVacunacion PresentaCarneVacunacionT{tomaReal}
                ,ControlesCrecimDesarrollo ControlesCrecimDesarrolloT{tomaReal}
            from baseModelo_t1
            where NoToma = {tomaReal}
           """) 
        print(f"-- Paso1: Creacion dataframe Fin ✓✓ ")
        print(f"-- Paso2: Creacion vista inicio ✓✓ ")
        globals()["Toma" + str(tomaReal)].createOrReplaceTempView(f"{tableName}")
        print(f"-- Paso2: Creacion vista inicio ✓✓ ")

-------- Inicio creacion tabla NoToma: 2--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 3--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 4--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 5--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 6--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 7--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 8--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 9--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 10--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 11--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 12--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 13--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 14--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 15--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 16--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-------- Inicio creacion tabla NoToma: 17--------
-- Paso1: Creacion dataframe Inicio -- 
-- Paso1: Creacion dataframe Fin ✓✓ 
-- Paso2: Creacion vista inicio ✓✓ 
-- Paso2: Creacion vista inicio ✓✓

Se toman los datos del DF1: 
> Datos Beneficiario + Datos Toma 1 + JOIN por cada una de las demás tomas nutricionales.

In [0]:
baseModeloSabana = spark.sql("""
select 
    IdBeneficiario,
    CodDocumento,
    NumeroDocumento,
    PrimerNombre,
    SegundoNombre,
    PrimerApellido,
    SegundoApellido,
    FechaNacimientoBeneficiario,
    sexo as sexoBeneficiario,
    NombreSexo,
    RegimenSeguridadSocial,
    CodPaisNacimientoBeneficiario,
    NombPaisNacimientoBeneficiario,
    CodPaisRasidenciaBen,
    NombPaisRasidenciaBen,
    CodDepartamentoResidenciaBen,
    NomDepartamentoResidenciaBen,
    CodMunicipResidenciaBen,
    NomMunicipResidenciaBen,
    CodComunaResidenciaBen,
    NomComunaResidenciaBen,
    CodBarrioResidenciaBen,
    NomBarrioResidenciaBen,
    CodVeredaResidenciaBen,
    NomVeredaResidenciaBen,
    CodRancheriaResidenciaBen,
    NomRancheriaResidenciaBen,
    PresentaDiscapacidad,
    ZonaUbicacionBeneficiario,
    GrupoEtnico,
    MesesLactanciaMaternaExclusiva,
    MesesLactanciaMaternaTotal,
    PesoBeneficiarioAlNacer,
    TallaBeneficiarioAlNacer,
    AntecendentePremadurez,
    EdadGestacionarAlNacer,
------------------------------------------------
    Toma TomaT1,
    NoToma NoTomaT1,
    IdDireccionesICBF IdDireccionesICBFT1,
    DireccionICBF DireccionICBFT1,
    NombreServicio NombreServicioT1,
    EdadBeneficiarioDias EdadBeneficiarioDiasT1,
    FechaRegistroSaludNutricion FechaRegistroSaludNutricionT1,
    FechaMedicionPerimetroBraquial FechaMedicionPerimetroBraquialT1,
    MedicionPerimetroBraquial MedicionPerimetroBraquialT1,
    peso pesoT1,
    Talla TallaT1,
    ZScoreTallaEdad ZScoreTallaEdadT1,
    ZScorePesoEdad ZScorePesoEdadT1,
    ZScorePesoTalla ZScorePesoTallaT1,
    ZScoreIMC ZScoreIMCT1,
    EstadoTallaEdad EstadoTallaEdadT1,
    EstadoPesoEdad EstadoPesoEdadT1,
    EstadoPesoTalla EstadoPesoTallaT1,
    EstadoIMC EstadoIMCT1,
    Flag FlagT1,
    PresentaCarneVacunacion PresentaCarneVacunacionT1,
    ControlesCrecimDesarrollo ControlesCrecimDesarrolloT1
------------------------------------------------
  ,t2.*
  ,t3.*
  ,t4.*
  ,t5.*
  ,t6.*
  ,t7.*
  ,t8.*
  ,t9.*
  ,t10.*
  ,t11.*
  ,t12.*
  ,t13.*
  ,t14.*
  ,t15.*
  ,t16.*
  ,t17.*
from baseModelo_t1 t1
    left join toma2 t2 on t1.idbeneficiario = t2.idbeneficiariot2
    left join toma3 t3 on t1.idbeneficiario = t3.idbeneficiariot3
    left join toma4 t4 on t1.idbeneficiario = t4.idbeneficiariot4
    left join toma5 t5 on t1.idbeneficiario = t5.idbeneficiariot5
    left join toma6 t6 on t1.idbeneficiario = t6.idbeneficiariot6
    left join toma7 t7 on t1.idbeneficiario = t7.idbeneficiariot7
    left join toma8 t8 on t1.idbeneficiario = t8.idbeneficiariot8
    left join toma9 t9 on t1.idbeneficiario = t9.idbeneficiariot9
    left join toma10 t10 on t1.idbeneficiario = t10.idbeneficiariot10
    left join toma11 t11 on t1.idbeneficiario = t11.idbeneficiariot11
    left join toma12 t12 on t1.idbeneficiario = t12.idbeneficiariot12
    left join toma13 t13 on t1.idbeneficiario = t13.idbeneficiariot13
    left join toma14 t14 on t1.idbeneficiario = t14.idbeneficiariot14
    left join toma15 t15 on t1.idbeneficiario = t15.idbeneficiariot15
    left join toma16 t16 on t1.idbeneficiario = t16.idbeneficiariot16
    left join toma17 t17 on t1.idbeneficiario = t17.idbeneficiariot17
where t1.NoToma = 1
""")
baseModeloSabana.createOrReplaceTempView("baseModeloSabana")

In [0]:
len(baseModeloSabana.columns)

Out[29]: 426

In [0]:
%sql
SELECT * FROM baseModeloSabana limit 10

IdBeneficiario,CodDocumento,NumeroDocumento,PrimerNombre,SegundoNombre,PrimerApellido,SegundoApellido,FechaNacimientoBeneficiario,sexoBeneficiario,NombreSexo,RegimenSeguridadSocial,CodPaisNacimientoBeneficiario,NombPaisNacimientoBeneficiario,CodPaisRasidenciaBen,NombPaisRasidenciaBen,CodDepartamentoResidenciaBen,NomDepartamentoResidenciaBen,CodMunicipResidenciaBen,NomMunicipResidenciaBen,CodComunaResidenciaBen,NomComunaResidenciaBen,CodBarrioResidenciaBen,NomBarrioResidenciaBen,CodVeredaResidenciaBen,NomVeredaResidenciaBen,CodRancheriaResidenciaBen,NomRancheriaResidenciaBen,PresentaDiscapacidad,ZonaUbicacionBeneficiario,GrupoEtnico,MesesLactanciaMaternaExclusiva,MesesLactanciaMaternaTotal,PesoBeneficiarioAlNacer,TallaBeneficiarioAlNacer,AntecendentePremadurez,EdadGestacionarAlNacer,TomaT1,NoTomaT1,IdDireccionesICBFT1,DireccionICBFT1,NombreServicioT1,EdadBeneficiarioDiasT1,FechaRegistroSaludNutricionT1,FechaMedicionPerimetroBraquialT1,MedicionPerimetroBraquialT1,pesoT1,TallaT1,ZScoreTallaEdadT1,ZScorePesoEdadT1,ZScorePesoTallaT1,ZScoreIMCT1,EstadoTallaEdadT1,EstadoPesoEdadT1,EstadoPesoTallaT1,EstadoIMCT1,FlagT1,PresentaCarneVacunacionT1,ControlesCrecimDesarrolloT1,IdBeneficiarioT2,TomaT2,NoTomaT2,IdDireccionesICBFT2,DireccionICBFT2,NombreServicioT2,EdadBeneficiarioDiasT2,FechaRegistroSaludNutricionT2,FechaMedicionPerimetroBraquialT2,MedicionPerimetroBraquialT2,pesoT2,TallaT2,ZScoreTallaEdadT2,ZScorePesoEdadT2,ZScorePesoTallaT2,ZScoreIMCT2,EstadoTallaEdadT2,EstadoPesoEdadT2,EstadoPesoTallaT2,EstadoIMCT2,FlagT2,PresentaCarneVacunacionT2,ControlesCrecimDesarrolloT2,IdBeneficiarioT3,TomaT3,NoTomaT3,IdDireccionesICBFT3,DireccionICBFT3,NombreServicioT3,EdadBeneficiarioDiasT3,FechaRegistroSaludNutricionT3,FechaMedicionPerimetroBraquialT3,MedicionPerimetroBraquialT3,pesoT3,TallaT3,ZScoreTallaEdadT3,ZScorePesoEdadT3,ZScorePesoTallaT3,ZScoreIMCT3,EstadoTallaEdadT3,EstadoPesoEdadT3,EstadoPesoTallaT3,EstadoIMCT3,FlagT3,PresentaCarneVacunacionT3,ControlesCrecimDesarrolloT3,IdBeneficiarioT4,TomaT4,NoTomaT4,IdDireccionesICBFT4,DireccionICBFT4,NombreServicioT4,EdadBeneficiarioDiasT4,FechaRegistroSaludNutricionT4,FechaMedicionPerimetroBraquialT4,MedicionPerimetroBraquialT4,pesoT4,TallaT4,ZScoreTallaEdadT4,ZScorePesoEdadT4,ZScorePesoTallaT4,ZScoreIMCT4,EstadoTallaEdadT4,EstadoPesoEdadT4,EstadoPesoTallaT4,EstadoIMCT4,FlagT4,PresentaCarneVacunacionT4,ControlesCrecimDesarrolloT4,IdBeneficiarioT5,TomaT5,NoTomaT5,IdDireccionesICBFT5,DireccionICBFT5,NombreServicioT5,EdadBeneficiarioDiasT5,FechaRegistroSaludNutricionT5,FechaMedicionPerimetroBraquialT5,MedicionPerimetroBraquialT5,pesoT5,TallaT5,ZScoreTallaEdadT5,ZScorePesoEdadT5,ZScorePesoTallaT5,ZScoreIMCT5,EstadoTallaEdadT5,EstadoPesoEdadT5,EstadoPesoTallaT5,EstadoIMCT5,FlagT5,PresentaCarneVacunacionT5,ControlesCrecimDesarrolloT5,IdBeneficiarioT6,TomaT6,NoTomaT6,IdDireccionesICBFT6,DireccionICBFT6,NombreServicioT6,EdadBeneficiarioDiasT6,FechaRegistroSaludNutricionT6,FechaMedicionPerimetroBraquialT6,MedicionPerimetroBraquialT6,pesoT6,TallaT6,ZScoreTallaEdadT6,ZScorePesoEdadT6,ZScorePesoTallaT6,ZScoreIMCT6,EstadoTallaEdadT6,EstadoPesoEdadT6,EstadoPesoTallaT6,EstadoIMCT6,FlagT6,PresentaCarneVacunacionT6,ControlesCrecimDesarrolloT6,IdBeneficiarioT7,TomaT7,NoTomaT7,IdDireccionesICBFT7,DireccionICBFT7,NombreServicioT7,EdadBeneficiarioDiasT7,FechaRegistroSaludNutricionT7,FechaMedicionPerimetroBraquialT7,MedicionPerimetroBraquialT7,pesoT7,TallaT7,ZScoreTallaEdadT7,ZScorePesoEdadT7,ZScorePesoTallaT7,ZScoreIMCT7,EstadoTallaEdadT7,EstadoPesoEdadT7,EstadoPesoTallaT7,EstadoIMCT7,FlagT7,PresentaCarneVacunacionT7,ControlesCrecimDesarrolloT7,IdBeneficiarioT8,TomaT8,NoTomaT8,IdDireccionesICBFT8,DireccionICBFT8,NombreServicioT8,EdadBeneficiarioDiasT8,FechaRegistroSaludNutricionT8,FechaMedicionPerimetroBraquialT8,MedicionPerimetroBraquialT8,pesoT8,TallaT8,ZScoreTallaEdadT8,ZScorePesoEdadT8,ZScorePesoTallaT8,ZScoreIMCT8,EstadoTallaEdadT8,EstadoPesoEdadT8,EstadoPesoTallaT8,EstadoIMCT8,FlagT8,PresentaCarneVacunacionT8,Cont

Esta validación es con el objetivo de asegurar que existan tantos registros como beneficarios únicos.

In [0]:
%sql
SELECT 
  COUNT(1) as TotalRegistrosBase,                                 -- Total de Registros 4392521
  COUNT(DISTINCT(IdBeneficiario)) as TotalBeneficiariosBase       -- Total de Beneficiarios Únicos 4392521
FROM baseModeloSabana  

TotalRegistrosBase,TotalBeneficiariosBase
4392522,4392522


En éste punto ya tenemos la base de los beneficiarios con las observaciones en columnas, es decir beneficiario + t1 + t2 + ... + tn.  
> Vamos a depurar la base de SISBEN puesto que hay N. Documento que corresponde a más de una persona (Nombres + Apellidos).   
> Se genera una llave y se establece un row_number() para generar el conteo de las observaciones ordenadas por fecha: Concatenación de documen + ape1 (Documento + Apellido 1) por cada observación. Esto pues hay números de documento que corresponden a más de una persona.

In [0]:
len(baseModeloSabana.columns)

Out[30]: 426

In [0]:
SisbenPartition = spark.sql("""
select 
    * 
from (
    select 
      *,
      row_number() over (partition by concat(num_documento,UPPER(pri_apellido)) order by fec_nacimiento desc) NumeroObs
    from sisben
    )
where NumeroObs = 1
""")
SisbenPartition.createOrReplaceTempView("SisbenPartition")

Acá evidenciamos que sí toma la llave y deja un registro único con la llave (Documento + Apellido 1)

In [0]:
%sql
SELECT                                                            
  COUNT(1) as TotalRegistrosBase,                                 -- Total de Registros 4392522 desde la Base Cuéntame
  COUNT(DISTINCT(IdBeneficiario)) as TotalBeneficiariosBase       -- Total de Beneficiarios Únicos 4392522 con INS y Sisben  IV
FROM baseModeloSabana 

Se integran las fuentes de:

> Base de modelo: RubOnline, Cuéntame

> Sisben: Base del SISBEN

> INS Cortes: Cortes del INS

# Cruce sabanas Sisben4 bm e INS sabana final

In [0]:
baseprueb = spark.sql("""
SELECT * FROM (
    SELECT
      bm.*,
      sis.*,
      ins.*,
      row_number() over (partition by idBeneficiario order by bm.FechaNacimientoBeneficiario desc) NumeroObsValu2
    FROM baseModeloSabana bm -- 110.425 Beneficiarios
    -- 
     
    --Acá cruzamos con INS:
    LEFT JOIN ins_cortes ins ON 
      CAST(ins.INS_num_ide_ as Int) = CAST(bm.NumeroDocumento as Int)  -- 110.425 Beneficiarios  -- 325 registros que cruzan con INS -- ahora son 372. Se validó el cruce OK. 

    --Acá cruzamos con SISBEN el (Documento + Apellido 1) de acuerdo a la regla que se planteó arriba:
    LEFT JOIN SisbenPartition sis on 
      CAST(sis.num_documento as Int) = CAST(bm.NumeroDocumento as Int)    
      AND UPPER(sis.pri_apellido) = UPPER(bm.PrimerApellido) -- 110.425 Beneficiarios -- 6.783 registros que cruzan con SISBEN ahora son 53.180 con la nueva llave. Se validó el cruce OK.
    
    
) WHERE NumeroObsValu2 = 1
""")
baseprueb.createOrReplaceTempView("baseprueb")

In [0]:
print(len(baseprueb))

#  se guarda la sabana final en formato delta

In [0]:
#---------------------------- Borrar Histórico ----------------------------------------#
#dbfsWriteDelta2 = "dbfs:/dbfs/gold"
dbfsWriteDelta2="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticosabana6/"
try:
    dbutils.fs.ls(dbfsWriteDelta2)
    print(" *** Borrando Directorio *** ")
    files = dbutils.fs.ls(dbfsWriteDelta2)
    for f in files:
        f.path
        dbutils.fs.rm(f.path, recurse=True)
except:
    print(" *** No Existe Directorio *** ")
    pass
#---------------------------- Directorio de Salida ------------------------------------#
print(" *** Guardando en el DataLake *** ")
pathSave = dbfsWriteDelta2
nombreTabla = "dbo.baseprueb"

baseprueb.write \
.format("delta") \
.mode("overwrite") \
.option("overwriteSchema", "true") \
.save(f"{pathSave}") 
print(" *** Guardado en el DataLake *** ")
spark.sql(f"CREATE TABLE IF NOT EXISTS {nombreTabla} USING DELTA LOCATION '{pathSave}/'")

*** Borrando Directorio *** 
 *** Guardando en el DataLake ***